In [48]:
#Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn import metrics, tree
from sklearn.model_selection import cross_val_score, cross_validate #método para evaluar varios particionamientos de C-V
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import RFECV
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
import math
import seaborn as sns

#Bagging
from sklearn.ensemble import BaggingClassifier

#Random Forest
from sklearn.ensemble import RandomForestClassifier

#Boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import zero_one_loss

In [49]:
data = pd.read_csv('data_set_cleaned.csv', header = 0, names=None)
data.head()

,Case1_Control0,Edad,Genero,Cycle Time,Stance Percent,Stance Time,Step Length,Step Number,Step Time,Step Cadence,...,Step Timel,Step Cadencel,Stride Numberl,Stride Lengthl,Swing Percentl,Swing Timel,Distancel,Duration Timel,Speedl,Accelerationl
0,Positivo,79,M,0.803979,0.644524,1.327075,0.796338,2,0.335486,58.280750,...,0.312041,88.428185,1,0.185463,0.459888,0.624083,1.890646,1.357033,1.393220,1.026666
1,Positivo,79,M,0.826989,0.633677,1.326004,0.823259,2,0.352115,57.346196,...,0.429035,59.169468,1,1.198056,0.423096,0.858070,1.923441,2.028073,0.948408,0.467640
2,Positivo,79,M,0.789131,0.610778,1.219237,0.739892,2,0.359121,60.114133,...,0.405821,56.550877,1,1.059078,0.382493,0.811643,1.974094,2.121983,0.930306,0.438414
3,Positivo,68,M,1.193026,0.536687,1.263887,0.741654,2,0.476885,50.955888,...,0.435792,50.936203,1,1.085094,0.369960,0.871583,1.951583,2.355888,0.828385,0.351623
4,Positivo,68,M,1.045567,0.508093,1.061983,0.817604,2,0.446011,57.412502,...,0.358499,59.642470,1,1.189026,0.356362,0.716997,1.858186,2.011989,0.923557,0.459027


In [50]:
x = data.loc[: , ~data.columns.isin(['Case1_Control0'])]
x = pd.get_dummies(data=x)
x = x.values

y = data['Case1_Control0'].values

## Fine tuning

Se buscan los mejores parametros de encontrar el mejor desempeño de los modelos basados en arboles de decisión y validando los resultados usando 10-CV para evitar el overfit:

#### Bagging:

In [66]:
#Tunear modelo Bagging

#Parametros a tunear
param_grid = {
    'base_estimator__max_depth' : [2, 5, 10, 200],
    'base_estimator__min_samples_split' : [5,10,20],
    'n_estimators' : [10, 50, 100, 200],
    'max_samples' : [0.05, 0.2, 0.5, 0.7],
    'max_features' : [0.05, 0.2, 0.5, 0.7]
}

GS_BT = GridSearchCV(BaggingClassifier(tree.DecisionTreeClassifier()),
                     param_grid=param_grid, scoring='accuracy', cv=10)

GS_BT.fit(x, y)

print('Mejores parametros:\n', GS_BT.best_params_)
print('Mejor Exactitud:\n', GS_BT.best_score_)

Mejores parametros:
 {'base_estimator__max_depth': 10, 'base_estimator__min_samples_split': 5, 'max_features': 0.7, 'max_samples': 0.05, 'n_estimators': 50}
Mejor Exactitud:
 0.7087912087912088


**Mejores parametros:**  
 {'base_estimator__max_depth': 10, 'base_estimator__min_samples_split': 5, 'max_features': 0.7, 'max_samples': 0.05, 'n_estimators': 50}  
**Mejor Exactitud:**  
 0.7087912087912088

#### RandomForest:

In [67]:
# Random Forest

param_grid = {
    'n_estimators' : [10, 20, 40, 100],
    'max_depth' : [8, 10],
    'min_samples_split' : [10, 20, 25],
    'min_samples_leaf' : [1, 5, 8, 10],
    'min_impurity_decrease' : [0.0, 0.1]
}

GS_RF = GridSearchCV(RandomForestClassifier(criterion='entropy', max_features=('auto'),
                                            bootstrap=True, oob_score=True,
                                            random_state=1234, n_jobs=2, verbose=0),
                     param_grid=param_grid, scoring='accuracy', cv=10)

GS_RF.fit(x,y)

print('Mejores parametros:\n', GS_RF.best_params_)
print('Mejor Exactitud:\n', GS_RF.best_score_)

Mejores parametros:
 {'max_depth': 8, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 100}
Mejor Exactitud:
 0.6758241758241759


**Mejores parametros:**  
 {'max_depth': 8, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 100}  
**Mejor Exactitud:**  
 0.6758241758241759

In [68]:
param_grid = {
    'n_estimators' : [randint.rvs(10,120,1)],
    'max_depth' : [randint.rvs(5,15,1)],
    'min_samples_split' : [randint.rvs(10,25,1)],
    'min_samples_leaf' : [randint.rvs(1,20,1)],
    'min_impurity_decrease' : uniform(0.0,0.1)
}

RS_RF = RandomizedSearchCV(RandomForestClassifier(criterion='entropy', max_features=('auto'),
                                            bootstrap=True, oob_score=True,
                                            random_state=1234, n_jobs=1, verbose=0), 
                           param_distributions=param_grid, scoring='accuracy', cv=10, random_state=1234)  

RS_RF.fit(x,y)

print('Mejores parametros:\n', RS_RF.best_params_)
print('Mejor Exactitud:\n', RS_RF.best_score_)

Mejores parametros:
 {'max_depth': 12, 'min_impurity_decrease': 0.06221087710398319, 'min_samples_leaf': 16, 'min_samples_split': 13, 'n_estimators': 95}
Mejor Exactitud:
 0.6703296703296703


**Mejores parametros:**  
 {'max_depth': 12, 'min_impurity_decrease': 0.06221087710398319, 'min_samples_leaf': 16, 'min_samples_split': 13, 'n_estimators': 95}  
**Mejor Exactitud:**  
 0.6703296703296703

#### Boosting (Adaptive Boosting)

Debido a que este modelo obtuvo un buen desempeño con el data set inicial, se usa posteriormente el dataset sin eliminar las anomalias, para explorar si los datos anomalos ayudar a discriminar los pacientes.

In [69]:
# Boosting

param_grid ={
    'n_estimators' : [10, 20, 100, 400],
    'learning_rate' : [0.05, 0.1, 0.2, 0.5,0.7,0.8]
}

GS_Boo = GridSearchCV(AdaBoostClassifier(base_estimator=None, algorithm='SAMME'), 
                      param_grid=param_grid, scoring='accuracy', cv=10)
GS_Boo.fit(x,y)

print('Mejores parametros:\n', GS_Boo.best_params_)
print('Mejor Exactitud:\n', GS_Boo.best_score_)

Mejores parametros:
 {'learning_rate': 0.05, 'n_estimators': 10}
Mejor Exactitud:
 0.7252747252747253


**Mejores parametros:**  
 {'learning_rate': 0.05, 'n_estimators': 10}   
**Mejor Exactitud:**  
 0.7252747252747253

In [13]:
data_a = pd.read_csv('data_set_with_anomalys.csv', header = 0, names=None)
data_a.head()

xa = data_a.loc[: , ~data_a.columns.isin(['Case1_Control0'])]
xa = pd.get_dummies(data=xa)
xa = xa.values

ya = data_a['Case1_Control0'].values

In [58]:
# Boosting with anomalys

param_grid ={
    'n_estimators' : [10, 20, 100, 400],
    'learning_rate' : [0.05, 0.1, 0.2, 0.5,0.7,0.8]
}

GS_Boo = GridSearchCV(AdaBoostClassifier(base_estimator=None, algorithm='SAMME'), 
                      param_grid=param_grid, scoring='accuracy', cv=10)
GS_Boo.fit(xa,ya)

print('Mejores parametros:\n', GS_Boo.best_params_)
print('Mejor Exactitud:\n', GS_Boo.best_score_)

Mejores parametros:
 {'learning_rate': 0.05, 'n_estimators': 10}
Mejor Exactitud:
 0.7252747252747253


**Mejores parametros:**  
 {'learning_rate': 0.05, 'n_estimators': 10}  
**Mejor Exactitud:**  
 0.7252747252747253 

## Recursive Feature Elimination (RFE)

In [64]:
ada = AdaBoostClassifier(n_estimators=20, learning_rate=0.05, algorithm='SAMME', base_estimator=None)

rfecv = RFECV(estimator=ada, step=1, cv=10, scoring='accuracy')

rfecv.fit(x,y)

print("Caracteristicas seleccionadas:",data.columns[rfecv.support_])
print("Mejor Exactitud:", np.max(rfecv.grid_scores_))

Caracteristicas seleccionadas: Index(['Cycle Timel', 'Step Timel'], dtype='object')
Mejor Exactitud: 0.7266081871345029


Caracteristicas seleccionadas: Index(['Cycle Timel', 'Step Timel'], dtype='object')  
Mejor Exactitud: 0.7266081871345029

In [65]:
# RFE with anomalys
ada = AdaBoostClassifier(n_estimators=10, learning_rate=0.05, algorithm='SAMME', base_estimator=None)

rfecv = RFECV(estimator=ada, step=1, cv=10, scoring='accuracy')

rfecv.fit(xa,ya)

print("Caracteristicas seleccionadas:",data.columns[rfecv.support_])
print("Mejor Exactitud:",  np.max(rfecv.grid_scores_))

Caracteristicas seleccionadas: Index(['Case1_Control0', 'Edad', 'Genero', 'Cycle Time', 'Stance Percent',
       'Stance Time', 'Step Length', 'Step Number', 'Step Time',
       'Step Cadence', 'Stride Number', 'Stride Length', 'Swing Percent',
       'Swing Time', 'Distance', 'Duration Time', 'Speed', 'Acceleration',
       'Cycle Timel', 'Stance Percentl', 'Stance Timel', 'Step Lengthl',
       'Step Numberl', 'Step Timel', 'Step Cadencel', 'Stride Numberl',
       'Stride Lengthl', 'Swing Percentl', 'Swing Timel', 'Distancel',
       'Duration Timel', 'Speedl', 'Accelerationl'],
      dtype='object')
Mejor Exactitud: 0.7266081871345029


**Caracteristicas seleccionadas:**   
Index(['Case1_Control0', 'Edad', 'Genero', 'Cycle Time', 'Stance Percent',
       'Stance Time', 'Step Length', 'Step Number', 'Step Time',
       'Step Cadence', 'Stride Number', 'Stride Length', 'Swing Percent',
       'Swing Time', 'Distance', 'Duration Time', 'Speed', 'Acceleration',
       'Cycle Timel', 'Stance Percentl', 'Stance Timel', 'Step Lengthl',
       'Step Numberl', 'Step Timel', 'Step Cadencel', 'Stride Numberl',
       'Stride Lengthl', 'Swing Percentl', 'Swing Timel', 'Distancel',
       'Duration Timel', 'Speedl', 'Accelerationl'],
      dtype='object')  
**Mejor Exactitud:**  
0.7266081871345029

Con el procedimiento RFE, se logra mejorar el desempeño de ambos dataset (limpio y original), pero, el dataset sin los datos anomalos logró mejorar la exactitud con solo dos caracteristicas (Columnas), mientras que el dataset original requiere de casi todas las columnas.

## Evaluación de metricas del modelo seleccionado (Mejor exactitud)

Métricas:
- **Accuracy:** porcentaje de personas bien clasificadas (TP y TF) de todos las personas. Esta métrica se puede usar ya que hay un buen balance de cada clase en el dataset.
- **Kappa:** Especifica la mejora del modelo con respecto a un modelo (Baseline) que "adivina" la clase (diagnóstico) de acuerdo a la frecuencia de cada clase (PD o No-PD).
- **Especificidad:** Proporción de pacientes clasificados correctamente como no-PD, de todas las personas sin PD.
- **Precisión:** Dice la proporción de pacientes clasificados correctamente con PD, del total de pacientes clasificados con PD.
- **Recall:** Proporción de pacientes clasificados correctamente con PD, del total pacientes diagnosticados con PD.
- **f1-Score:** Es la media armónica entre la precisión y el recall, lo cual entrega una métrica que permite evaluar P y R. La media armónica puede representar mejor al recall y la preción cuando existe mucha diferencia entre ambos.

In [57]:
def wrap_precision(y_test,y_pred):
    return metrics.precision_score(y_test,y_pred,pos_label='Positivo')

def wrap_recall(y_test,y_pred):
    return metrics.recall_score(y_test,y_pred,pos_label='Positivo')

def wrap_f1(y_test,y_pred):
    return metrics.f1_score(y_test,y_pred,pos_label='Positivo')

def wrap_kappa(y_test,y_pred):
    return metrics.cohen_kappa_score(y_test,y_pred)

In [71]:
scores = {
    'acc':'accuracy', 
    'kappa' : metrics.make_scorer(wrap_kappa),
    'precision' : metrics.make_scorer(wrap_precision),
    'recall' : metrics.make_scorer(wrap_recall),
    'f1' : metrics.make_scorer(wrap_f1)
}

cv_scores = cross_validate(estimator=ada, X=x,y=y, cv=10, return_train_score=False, scoring=scores)

print('Exactitud: ', np.average(cv_scores['test_acc']))
print('Kappa_Score: ', np.average(cv_scores['test_kappa']))
print()
print('Precisión: ', np.average(cv_scores['test_precision']))
print('Recall: ', np.average(cv_scores['test_recall']))
print('F1_score: ', np.average(cv_scores['test_f1']))

Exactitud:  0.7266081871345029
Kappa_Score:  0.4534192756292204

Precisión:  0.748088023088023
Recall:  0.6888888888888889
F1_score:  0.7054295051353876


#### Interpretación de métricas:

La exactitud del modelo AdaBoost y la regresión logistica son muy similares, pero este modelo tiene una mejor sensibilidad que la regresión, lo cual indica que es mejor para diferenciar a los pacientes con PD de los pacientes sanos, a pesar de que sacrifique un poco la precisión.

Por lo anterior, el modelo de AdaBoost es la primera opción para diagnósticar pacientes de PD de los modelo explorados.